Data Fetching from Yahoo Finacial API<br>
Data preprocessing and encoding<br>
Model Define<br>

<h1>Model Inputs:</h1>
<li>Baisc Securites Information</li>
<li>Historical Stock value</li>
<li>News (National News and World News Headlines)</li>
<h1>Model Output</h1>
<p>Stock Prediction for 1 day in 1 hour interval
[ Monday through Friday from 9:15 AM to 3:30 PM India Standard Time (GMT+05:30) ]

Shape of Output: [6, 7]</p>

In [ ]:
!pip install gnewsclient
!pip install transformers
!pip install tqdm boto3 requests regex sentencepiece sacremoses python-Levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6047 sha256=e7c9728e13c629b816ace8054261def216e4ac1141af55946a652590d5dc6f6a
  Stored in directory: /root/.cache/pip/wheels/f0/69/93/a47e9d621be168e9e33c7ce60524393c0b92ae83cf6c6e89c5
Successfully built sgmllib3k
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 38.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<h1>Loading symbols of listed stock in NSE</h1>

In [ ]:
# get list of symbols available
import pandas as pd

symbols = list()
list_loc = r"/content/drive/MyDrive/mtech_project/stock_data/stock_list.txt"
with open(list_loc, 'r') as f:
  for line in f.readlines():
    symbols.append(line.strip()+'.NS')



print(len(symbols), symbols[0], symbols[len(symbols)-1])

401 3IINFOLTD.NS ZOTA.NS


<h1>Loading and defineing tokenizer and Model</h1>

In [ ]:
from torch import nn
class LinerLayer(nn.Module):
    def __init__(self, BATCH_SIZE, SEQ_LEN, HIDDEN_LEN, OUTPUT):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Flatten(start_dim = 1),
            nn.Linear(SEQ_LEN*HIDDEN_LEN, OUTPUT),
            nn.ReLU(),
            nn.Flatten(start_dim = 0),
            nn.Linear(BATCH_SIZE*OUTPUT, OUTPUT),
            nn.ReLU()
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

BATCH_SIZE = 16
SEQ_LEN = 512
HIDDEN_LEN = 768
OUTPUT = 42

import torch
model = LinerLayer(BATCH_SIZE, SEQ_LEN, HIDDEN_LEN, OUTPUT)
input = torch.rand(BATCH_SIZE, SEQ_LEN, HIDDEN_LEN)
model(input)

tensor([0.0091, 0.1345, 0.1129, 0.0000, 0.0000, 0.0432, 0.0055, 0.1118, 0.0000,
        0.0000, 0.0000, 0.0545, 0.0000, 0.0613, 0.1554, 0.0236, 0.0000, 0.0000,
        0.0000, 0.1133, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.1219, 0.0000, 0.0000, 0.0324, 0.0829, 0.1795, 0.0939, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0311, 0.0000, 0.1998, 0.0000],
       grad_fn=<ReluBackward0>)

In [ ]:
out  = model(input)
del model

In [ ]:
import torch
from torch import nn

BATCH_SIZE = 64
SEQ_LEN = 512
HIDDEN_LEN = 768
OUTPUT = 49

PATH = r'/content/drive/MyDrive/mtech_project/stock_data/model/StockPredictionBERT.pt'

tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'bert-base-cased')
model = torch.hub.load('huggingface/pytorch-transformers', 'model', 'bert-base-cased')

# appending linear layer at the end BERT
reg_layer = LinerLayer(BATCH_SIZE, SEQ_LEN, HIDDEN_LEN, OUTPUT)

model.add_module('lin_layer', reg_layer)


torch.save(model, PATH)
# Save the model in local drive


/usr/local/lib/python3.10/dist-packages/torch/hub.py:286: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/huggingface/pytorch-transformers/zipball/main" to /root/.cache/torch/hub/main.zip


Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_main


In [ ]:
PATH = r'/content/drive/MyDrive/mtech_project/stock_data/model/StockPredictionBERT_state_dict.pt'
torch.save(model.state_dict(), PATH)

<h1>Obtaine the data</h1>

In [ ]:
# Basis information by ticker name

import yfinance as yf

def get_fundamental_info(ticker_name):
  """Divide the input dictinary into multiple seq with specified length.

  Args:
    ticker_name: A SYMBOL.

  Returns:
    info: A dict with basic information about the given symbol.
    hist: historical market data of period 1 day and interval of 1 min
  """
  # get basic data
  ticker = yf.Ticker(ticker_name)

  # get historical market data
  hist = ticker.history(period="1d", interval="1m")

  return ticker.info, hist

In [ ]:
get_fundamental_info('ZOTA.NS')

({'address1': 'Zota House',
  'address2': '2/896, Hira Modi Street Sagrampura',
  'city': 'Surat',
  'zip': '395002',
  'country': 'India',
  'phone': '91 26 1233 1601',
  'website': 'https://www.zotahealthcare.com',
  'industry': 'Biotechnology',
  'industryKey': 'biotechnology',
  'industryDisp': 'Biotechnology',
  'sector': 'Healthcare',
  'sectorKey': 'healthcare',
  'sectorDisp': 'Healthcare',
  'longBusinessSummary': 'Zota Health Care Limited develops, manufactures, and markets pharmaceutical products in India and internationally. It offers generic drugs, over-the-counter products, ayurvedic products, and nutraceutical products. The company provides various medicines for chronic illnesses, including diabetes, heart conditions, and thyroid disorders. It operates its retail stores under the DAVAINDIA name. The company also exports its products to approximately 30 countries. Zota Health Care Limited was founded in 1995 and is headquartered in Surat, India.',
  'fullTimeEmployees': 3

In [ ]:
# Get market news and parse it
from gnewsclient import gnewsclient

def get_national_news():
  client = gnewsclient.NewsClient(language='en', location='india', topic='Business', max_results=100)
  news = client.get_news() # list containes dict
  national_news = list()
  for title in news:
    national_news.append(title['title']+'. ')
  return national_news

# Get market news and parse it
def get_world_news():
  client = gnewsclient.NewsClient(language='en', location='india', topic='World', max_results=100)
  news = client.get_news() # list containes dict
  national_news = list()
  for title in news:
    national_news.append(title['title']+'. ')
  return national_news

news = get_national_news()
print("No of news: ", len(news), ", News: At Index[0] ", news[0], ", At Index[", len(news), "] ", news[len(news)-1])

No of news:  70 , News: At Index[0]  HDFC Bank Q2 earnings: Net profit rises 50% to Rs 15,976 crore - Moneycontrol.  , At Index[ 70 ]  IndianOil to invest Rs 1,660 cr in joint venture with NPTC - Moneycontrol. 


<h1>Data Preprocessing functions</h1>

In [ ]:
# Preprocess the data
def trim_string(string, length):
  """Trims a string to a specified length.

  Args:
    string: The string to trim.
    length: The desired length of the string.

  Returns:
    The trimmed string.
  """

  if len(string) <= length:
    return string
  else:
    return string[:length]

def symbol_info2_segment(dict1):
  """Divide the input dictinary into multiple seq with specified length.

  Args:
    dict1: A dictionary.

  Returns:
    A list with divided seq.
  """
  #print(dict1)
  tot_seq = list()
  for key, value in dict1.items():
    if (str(key) != 'companyOfficers'):
      tot_seq.append(str(key)+" is "+ trim_string(str(value), 800) + ". ")
  return tot_seq

def news_into_segment(news_list):
  """Divide the input news list into multiple seq with specified length.

  Args:
    dict1: A dictionary.

  Returns:
    A list with divided seq.
  """
  max_seq_len = 800
  seq = ""
  tot_seq = list()
  for news in news_list:
    length = len(news)

    if max_seq_len >= length:
      seq += news+". "
      max_seq_len -= length
    else:
      max_seq_len = 800
      tot_seq.append(seq)
      seq = ""
      seq += news+". "
      max_seq_len -= length
  del seq
  return tot_seq

def clean_text(text):
  text = text.replace(" ", "")
  text = text.replace('Open', 'Opening is ')
  text = text.replace('\n', ', ')
  text = text.replace('High', ' high is ')
  text = text.replace('Low', ' low is ')
  text = text.replace('Close', ' closing is ')
  text = text.replace('Volume', ' volume traded is ')
  text = text.replace('Dividends', ' dividends are ')
  text = text.replace('StockSplits', ' stock splits are ')
  return text.strip()

def hist_into_segment(hist):
  """Divide the input news list into multiple seq with specified length.

  Args:
    hist: A dataframe.

  Returns:
    A list with divided seq.
  """
  if hist is None:
    return torch.zeros(26, 7)
  seq_li = list()
  no_of_row = hist.shape[0]
  for i in range(no_of_row):
    seq_li.append(clean_text(hist.iloc[i].to_string()) + ", at " + str(hist.iloc[i].name)+". ")
  return seq_li

In [ ]:

# Gather all data
n_news = get_national_news()
w_news = get_world_news()

def get_data(symbol):
  info, hist = get_fundamental_info(symbol)

  input = list()
  for data in symbol_info2_segment(info):
    input.append(data)

  for data in hist_into_segment(hist):
    input.append(data)

  for data in n_news:
    input.append(data)

  for data in w_news:
    input.append(data)

  #print("Input Length", len(input))
  #print(type(input[0]))
  return input

def tokenize(input):
  indexed_tokens = list()
  len_counter = 512
  seq_part = list()
  for data in input:
    for token in tokenizer.encode(data):
      if len_counter == 0:
        indexed_tokens.append(seq_part)
        seq_part = list()
        len_counter = 511
        seq_part.append(token)
      else:
        seq_part.append(token)
        len_counter -= 1

  tokens_tensor = torch.tensor(indexed_tokens)
  a_shape = tokens_tensor.shape
  if tokens_tensor.size()[0] > BATCH_SIZE:
    tokens_tensor = tokens_tensor[:BATCH_SIZE]
  elif tokens_tensor.size()[0] < BATCH_SIZE:
    padding_size = BATCH_SIZE - tokens_tensor.size()[0]
    padding_t = torch.zeros(padding_size, tokens_tensor.size()[1], dtype=torch.int8)
    tokens_tensor = torch.cat((tokens_tensor, padding_t), dim=0)
  print('Input Tensor Size: before - ', a_shape, ', after - ', tokens_tensor.shape)
  return tokens_tensor


In [ ]:
pred_save_loc = r"/content/drive/MyDrive/mtech_project/stock_data/PREDICTION/"
print(pred_save_loc + 'NAME' +'.pkl')

/content/drive/MyDrive/mtech_project/stock_data/PREDICTION/NAME.pkl


In [ ]:
# Prediction and Training logic
# Criterian for GA and Adam Implimentation
# Ref: https://pytorch.org/tutorials/beginner/saving_loading_models.html

import pickle

pred_save_loc = r"/content/drive/MyDrive/mtech_project/stock_data/PREDICTION/"
# symbol.pkl

def save(path: str, prediction: dict):
  with open(path, "wb") as f:
    pickle.dump(prediction, f)

model.eval()
# update result
# people.update({"name": "John"})
count = 0
for symbol in symbols:
  count += 1
  input_tokens = tokenize(get_data(symbol))
  with torch.no_grad():
    output = model(input_tokens)
  save(pred_save_loc + symbol +'.pkl', output[0])
  print(symbol, 'Remaining: ', len(symbols) - count)

Input Tensor Size: before -  torch.Size([57, 512]) , after -  torch.Size([64, 512])
3IINFOLTD.NS Remaining:  400
Input Tensor Size: before -  torch.Size([54, 512]) , after -  torch.Size([64, 512])
5PAISA.NS Remaining:  399
Input Tensor Size: before -  torch.Size([49, 512]) , after -  torch.Size([64, 512])
AAATECH.NS Remaining:  398
Input Tensor Size: before -  torch.Size([30, 512]) , after -  torch.Size([64, 512])
AARTISURF.NS Remaining:  397
Input Tensor Size: before -  torch.Size([28, 512]) , after -  torch.Size([64, 512])
AARVI.NS Remaining:  396
Input Tensor Size: before -  torch.Size([70, 512]) , after -  torch.Size([64, 512])
AAVAS.NS Remaining:  395
Input Tensor Size: before -  torch.Size([67, 512]) , after -  torch.Size([64, 512])
ABCAPITAL.NS Remaining:  394
Input Tensor Size: before -  torch.Size([15, 512]) , after -  torch.Size([64, 512])
ACEINTEG.NS Remaining:  393
Input Tensor Size: before -  torch.Size([27, 512]) , after -  torch.Size([64, 512])
ACL.NS Remaining:  392
Inp